In [ ]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [ ]:
MAIN_FOLDER = main_folder("area51")

create_database('blp_dataset')
create_table('products_tmp_area51')

In [ ]:
folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\shop")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())

            if page_content is not None:
                ## GET DESCRIPTION
                description = ""
                raw_desc = page_content.find("li", attrs={"id": "productDescription"})
                
                if raw_desc is not None:
                    raw_desc.find("h2").decompose()

                    if raw_desc is not None:
                        description = get_content(raw_desc)
                
                ## GET TITLE
                title = ""
                raw_title = page_content.find("li", attrs={"id": "productTitle"})
                
                if raw_title is not None:
                    title = get_content(raw_title)

                ## GET SELLER
                seller = ""
                raw_seller = page_content.find("div", attrs={"id": "vendorViewWrapper"})
                
                if raw_seller is not None:
                    raw_seller = raw_seller.find("a")
                    
                    if raw_seller is not None:
                        seller = get_content(raw_seller)
                
                
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_area_51 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Armory", seller, "t")
                    
                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid

                    ## GET IMAGE
                    img_content = page_content.find("li", attrs={"id": "productThumb"})
                    
                    if img_content is not None:
                        img_content = img_content.find("a", attrs={"class", "lightbox"})
                        
                        if img_content is not None:
                            img = img_content.find("img")["src"]
                            img_path = Path(f"{folder_image}\{img}")
                            
                            if img_path.exists():
                                try:
                                    # copy images to new directory, renaming with product id
                                    print(f"Image {product_id} exists... creating")

                                    dst_dir = Path(f"D:\images\p_area_51\{product_id}.jpg")
                                    copyfile(img_path, dst_dir)
                                except:
                                    print(f"Image {product_id} has error. Except")
                                    update_image(product_id, "no_image")
                            else:
                                print(f"Product {product_id} path image not found")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Product {product_id} path image not found")
                            update_image(product_id, "no_image")  
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")